In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing.data import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F

# Hyperparameters
num_epochs = 30
num_classes = 2
learning_rate = 0.002

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
df = pd.read_csv("datasets/creditcardfraud/creditcard.csv", sep=",", index_col=None)
print(df.shape)
df.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df['Time'] = StandardScaler().fit_transform(df['Time'].values.reshape(-1, 1))
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,1.641931,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,-0.350151,0
284803,1.641952,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,-0.254117,0
284804,1.641974,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,-0.081839,0
284805,1.641974,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,-0.313249,0
284806,1.642058,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,0.514355,0


In [4]:
anomalies = df[df["Class"] == 1]
normal = df[df["Class"] == 0]

anomalies.shape, normal.shape


((492, 31), (284315, 31))

In [5]:
for f in range(0, 20):
    normal = normal.iloc[np.random.permutation(len(normal))]
    

data_set = pd.concat([normal[:10000], anomalies])

x_train, x_test = train_test_split(data_set, test_size = 0.4, random_state = 42)

x_train = x_train.sort_values(by=['Time'])
x_test = x_test.sort_values(by=['Time'])

y_train = x_train["Class"]
y_test = x_test["Class"]

x_train.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
8,-1.996436,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.019392,0
99,-1.995151,1.232996,0.189454,0.491040,0.633673,-0.511574,-0.990609,0.066240,-0.196940,0.075921,...,-0.251566,-0.770139,0.125998,0.369627,0.205598,0.094062,-0.033138,0.020990,-0.289300,0
118,-1.994983,-0.997176,0.228365,1.715340,-0.420067,0.560838,0.564725,0.846047,0.197491,-0.097202,...,-0.070241,0.015445,0.072651,-0.272272,-0.087682,0.138132,0.125902,-0.063022,-0.007675,0
177,-1.994182,1.194066,-0.072582,0.635286,0.768616,-0.735584,-0.673466,-0.146299,-0.065653,0.646048,...,-0.300561,-0.770991,0.074444,0.397713,0.285947,0.286760,-0.031033,0.017052,-0.257755,0
225,-1.993488,-2.687978,4.390230,-2.360483,0.360829,1.310192,-1.645253,2.327776,-1.727825,4.324752,...,-1.045961,-0.156951,0.079854,-0.012598,0.207194,-0.536578,0.950393,-0.624431,-0.349671,0
259,-1.992729,0.726749,-0.528042,0.050366,1.373621,-0.124122,0.415688,0.259555,0.085114,-0.003025,...,0.061888,-0.101838,-0.308373,-0.270673,0.604410,-0.333689,-0.003944,0.040685,0.546220,0
356,-1.991087,1.260328,0.299161,0.527681,0.614899,-0.420592,-0.977533,0.108485,-0.244502,-0.058316,...,-0.249401,-0.688429,0.128290,0.398967,0.240282,0.092213,-0.024633,0.020529,-0.345313,0
374,-1.990834,1.124355,-0.132953,0.588467,0.804871,-0.726266,-0.521875,-0.167010,0.059298,0.368093,...,-0.165166,-0.446871,0.017617,0.548845,0.347079,0.239948,-0.038564,0.006765,-0.223252,0
379,-1.990729,-1.092301,0.430750,1.249785,0.429757,1.272076,0.548203,-0.120592,0.452571,-0.414005,...,-0.078916,0.072870,0.126237,-0.648165,-1.049104,0.206696,0.411803,0.188800,-0.342475,0
400,-1.990476,-0.695818,0.581773,2.378180,0.063396,0.329119,-0.449865,1.269104,-0.758363,0.381712,...,-0.327948,-0.369683,-0.426987,0.420170,0.235207,0.194957,-0.708471,-0.621219,-0.231288,0


In [6]:
x_train = np.array(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = np.array(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

y_train = np.array(y_train).reshape(y_train.shape[0] , 1)
y_test = np.array(y_test).reshape(y_test.shape[0], 1)

print("Shapes:\nx_train:%s\ny_train:%s\n" % (x_train.shape, y_train.shape))
print("x_test:%s\ny_test:%s\n" % (x_test.shape, y_test.shape))

Shapes:
x_train:(6295, 1, 31)
y_train:(6295, 1)

x_test:(4197, 1, 31)
y_test:(4197, 1)



In [7]:
class TCN(nn.Module):
    def __init__(self):
        super(TCN, self).__init__()
        
        self.conv_1 = nn.Conv1d(1, 128, kernel_size=2, dilation=1, padding=((2-1) * 1))
        self.conv_2 = nn.Conv1d(128, 128, kernel_size=2, dilation=2, padding=((2-1) * 2))
        self.conv_3 = nn.Conv1d(128, 128, kernel_size=2, dilation=4, padding=((2-1) * 4))
        self.conv_4 = nn.Conv1d(128, 128, kernel_size=2, dilation=8, padding=((2-1) * 8))
        self.dense_1 = nn.Linear(31*128  , 128)
        self.dense_2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv_1(x)
        x = x[:, :, :-self.conv_1.padding[0]]
        x = F.relu(x)
        x = F.dropout(x, 0.05)
        x = self.conv_2(x)
        x = x[:, :, :-self.conv_2.padding[0]]
        x = F.relu(x)
        x = F.dropout(x, 0.05)
        x = self.conv_3(x)
        x = x[:, :, :-self.conv_3.padding[0]]
        x = F.relu(x)
        x = F.dropout(x, 0.05)
        x = self.conv_4(x)
        x = x[:, :, :-self.conv_4.padding[0]]
        x = F.relu(x)
        x = F.dropout(x, 0.05)
        x = x.view(-1, 31*128)
        x = F.relu(self.dense_1(x))
        x = self.dense_2(x)
        return F.log_softmax(x, dim=1)

In [8]:
def train(model, device, x_train, y_train, criterion, optimizer, epoch, save_dir='TCN_CreditCard_PyTorch.ckpt'):
    total_step = len(x_train)
    
    x_train = torch.Tensor(x_train).cuda().float()
    y_train = torch.Tensor(y_train).cuda().long()

    x_train.to(device)
    y_train.to(device)

    # Forward pass
    outputs = model(x_train)
    loss = criterion(outputs, y_train.squeeze(1))

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch {}/{}, Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

    torch.save(model.state_dict(), save_dir)

In [9]:
from sklearn.metrics import roc_auc_score

def test(model, device, x_test, y_test):
    preds = []
    y_true = []
    
    # Set model to evaluation mode.
    model.eval()  
    with torch.no_grad():
        correct = 0
        total = 0
        
        x_test = torch.Tensor(x_test).cuda().float()
        y_test = torch.Tensor(y_test).cuda().long()
        
        x_test = x_test.to(device)
        y_test = y_test.to(device)
        y_test = y_test.squeeze(1)
        outputs = model(x_test)
        _, predicted = torch.max(outputs.data, 1)
        total += y_test.size(0)
        correct += (predicted == y_test).sum().item()    
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = y_test.detach().cpu().numpy()
        for f in range(0, len(detached_label)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])

        print('Test Accuracy of the model on the 10000 test images: {:.2%}'.format(correct / total))

        preds = np.eye(num_classes)[preds]
        y_true = np.eye(num_classes)[y_true]  
        auc = roc_auc_score(np.round(preds), y_true)
        print("AUC: {:.2%}".format (auc))

In [15]:

model = TCN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Training phase

for epoch in range(0, num_epochs):
    train(model, device, x_train, y_train, criterion, optimizer, epoch)


Epoch 1/30, Loss: 0.7319
Epoch 2/30, Loss: 0.5128
Epoch 3/30, Loss: 0.3454
Epoch 4/30, Loss: 0.2430
Epoch 5/30, Loss: 0.1245
Epoch 6/30, Loss: 0.1041
Epoch 7/30, Loss: 0.0684
Epoch 8/30, Loss: 0.0737
Epoch 9/30, Loss: 0.0646
Epoch 10/30, Loss: 0.0672
Epoch 11/30, Loss: 0.0650
Epoch 12/30, Loss: 0.0545
Epoch 13/30, Loss: 0.0526
Epoch 14/30, Loss: 0.0428
Epoch 15/30, Loss: 0.0403
Epoch 16/30, Loss: 0.0409
Epoch 17/30, Loss: 0.0453
Epoch 18/30, Loss: 0.0366
Epoch 19/30, Loss: 0.0357
Epoch 20/30, Loss: 0.0358
Epoch 21/30, Loss: 0.0351
Epoch 22/30, Loss: 0.0347
Epoch 23/30, Loss: 0.0342
Epoch 24/30, Loss: 0.0333
Epoch 25/30, Loss: 0.0329
Epoch 26/30, Loss: 0.0322
Epoch 27/30, Loss: 0.0316
Epoch 28/30, Loss: 0.0314
Epoch 29/30, Loss: 0.0307
Epoch 30/30, Loss: 0.0304


In [16]:
## Testing phase

test(model, device, x_test, y_test)

Test Accuracy of the model on the 10000 test images: 99.14%
AUC: 98.98%
